In [8]:
#import dependencies
import pandas as pd
import numpy as py
import os
import sys
import glob
import warnings


import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [15]:
kaggle_list = []
path = 'DataFiles/*.csv'

for fname in glob.glob(path):
    kaggle_list.append(fname)

kaggle_list

['DataFiles\\Cities.csv',
 'DataFiles\\Conferences.csv',
 'DataFiles\\ConferenceTourneyGames.csv',
 'DataFiles\\GameCities.csv',
 'DataFiles\\NCAATourneyCompactResults.csv',
 'DataFiles\\NCAATourneyDetailedResults.csv',
 'DataFiles\\NCAATourneySeedRoundSlots.csv',
 'DataFiles\\NCAATourneySeeds.csv',
 'DataFiles\\NCAATourneySlots.csv',
 'DataFiles\\RegularSeasonCompactResults.csv',
 'DataFiles\\RegularSeasonDetailedResults.csv',
 'DataFiles\\Seasons.csv',
 'DataFiles\\SecondaryTourneyCompactResults.csv',
 'DataFiles\\SecondaryTourneyTeams.csv',
 'DataFiles\\TeamCoaches.csv',
 'DataFiles\\TeamConferences.csv',
 'DataFiles\\Teams.csv',
 'DataFiles\\TeamSpellings.csv']

In [42]:
kaggle_dict = {}

for file in kaggle_list:
    kaggle_dict[os.path.basename(file)] = pd.DataFrame(pd.read_csv(file, sep=";", encoding='cp1252'))

kaggle_dict

{'Cities.csv':               CityID,City,State
 0               4001,Abilene,TX
 1                 4002,Akron,OH
 2                4003,Albany,NY
 3           4004,Albuquerque,NM
 4             4005,Allentown,PA
 5                  4006,Ames,IA
 6               4007,Amherst,MA
 7               4008,Anaheim,CA
 8             4009,Anchorage,AK
 9             4010,Ann Arbor,MI
 10            4011,Annapolis,MD
 11            4012,Arlington,TX
 12            4013,Asheville,NC
 13               4014,Athens,GA
 14               4015,Athens,OH
 15              4016,Atlanta,GA
 16        4017,Atlantic City,NJ
 17               4018,Auburn,AL
 18         4019,Auburn Hills,MI
 19               4020,Austin,TX
 20          4021,Bakersfield,CA
 21            4022,Baltimore,MD
 22               4023,Bangor,ME
 23          4024,Baton Rouge,LA
 24              4025,Bayamon,PR
 25             4026,Beaumont,TX
 26             4027,Berkeley,CA
 27            4028,Bethlehem,PA
 28             4029,Billings